## Лабораторная работа 7.1

In [1]:
!pip install xlwings

In [2]:
import pandas as pd
import xlwings as xw
import numpy as np

Лабораторная таблицы

1. Загрузите данные из файлов reviews_short.csv и recipes_short_5.csv в виде pd.DataFrame. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: id, name, minutes, submitted, description, n_ingredients

In [3]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0, encoding='utf8')
reviews

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...
1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [4]:
recipes = pd.read_csv('recipes_sample.csv', encoding='utf8').drop(columns=['contributor_id', 'n_steps'])
recipes

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл recipes.xlsx. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [5]:
recipes[559:560]

,name,id,minutes,submitted,description,n_ingredients
559,almost authentic cincinnati chili,12441,210,2001-10-05,9 @ from cincinnati and suffers pangs of withd...,14.0


In [6]:
with pd.ExcelWriter('recipes.xlsx') as excel_writer:
  recipes[:int(recipes.shape[0]*0.05)].to_excel(excel_writer, sheet_name='Рецепты')
  reviews[:int(reviews.shape[0]*0.05)].to_excel(excel_writer, sheet_name='Отзывы')

3. Используя xlwings, добавьте на лист Рецепты столбец seconds_assign, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [7]:
wb = xw.Book('recipes.xlsx')
sh1 = wb.sheets['Рецепты']

In [8]:
sh1.range('H1').value = 'seconds_assign'

In [9]:
sh1.range('H2:H1501').options(transpose=True).value = sh1.range('D2:D1501').options(np.array).value * 60

4. Используя xlwings, добавьте на лист Рецепты столбец seconds_formula, показывающий время выполнения рецепта в секундах. Выполните задание при формул Excel.

In [23]:
sh1.range('I1').value = 'seconds_formula'

In [24]:
sh1.range('I2:I1501').formula = '=D2*60'

5. Добавьте на лист Рецепты столбец n_reviews, содержащий кол-во отзывов для этого рецепта

In [25]:
sh1.range('J1').value = 'n_reviews'

In [26]:
recipes_reviews = pd.merge(recipes[:int(recipes.shape[0]*0.05)], reviews[:int(reviews.shape[0]*0.05)], left_on='id', right_on='recipe_id')
recipes_reviews.dropna(subset=('review',), inplace=True)
recipes_reviews.drop(['id', 'minutes', 'submitted',
       'description', 'n_ingredients',
       'user_id', 'date', 'rating', 'review'], axis=1, inplace=True)
recipes_reviews.head(1)

,name,recipe_id
0,healthy for them yogurt popsicles,67664


In [27]:
counter = recipes_reviews.groupby('recipe_id', as_index=False)['name'].count()
counter.columns = ['recipe_id', 'review_count']
counter.head(1)

,recipe_id,review_count
0,153,3


In [28]:
recipes_reviews = pd.merge(recipes_reviews, counter, on='recipe_id')
recipes_reviews.head(1)

,name,recipe_id,review_count
0,healthy for them yogurt popsicles,67664,1


In [15]:
for index, cell in enumerate(sh1.range('J2:J1501')):
    row_num = int(index) + 2
    recipe = int(sh1.range(f'C{row_num}').value)
    try:
        cell.value = recipes_reviews[recipes_reviews['recipe_id'] == recipe]['review_count'].iloc[0]
    except IndexError:
        cell.value = 0

6. Раскрасьте ячейки столбца minutes в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [16]:
for cell in sh1.range('D2:D1501'):
    cell.color = (0, 255, 0) if (value := cell.value) < 5 else (255, 255, 0) if value <= 10 else (255, 0, 0)

7. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [17]:
sh1.range('H1:J1').font.bold = True

In [18]:
sh1.range('H1:J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

8. Напишите функцию validate(), которая проверяет соответствие всех строк из листа interactions следующим правилам:
Рейтинг - это число от 0 до 5 включительно
Соответствующий рецепт имеется на листе рецептов
В случае несоответствия этим правилам, выделите строку красным цветом